In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import TransformerModel
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts.metrics import rmse, mae
import matplotlib.pyplot as plt

class OilTempForecastPipeline:
    def __init__(self, file_path, start_date='2023-09-01 00:00:00'):
        self.file_path = file_path
        self.start_date = pd.to_datetime(start_date)
        self.model = None

    def load_and_preprocess(self):
        df = pd.read_excel(self.file_path)
        df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
        df = df.dropna(subset=['DateTime'])
        df = df[df['DateTime'] >= self.start_date]
        df = df.set_index('DateTime').resample('1H').mean().interpolate(method='linear')
        self.df = df

    def build_series(self):
        self.series = TimeSeries.from_dataframe(self.df, value_cols='oil temperature', freq='h')
        self.covariates = TimeSeries.from_dataframe(self.df, value_cols=['winding temperature', 'tap position'], freq='h')

    def scale_and_split(self, train_ratio=0.95):
        self.scaler_series = Scaler(MinMaxScaler())
        self.series_scaled = self.scaler_series.fit_transform(self.series)

        self.scaler_cov = Scaler(MinMaxScaler())
        self.covariates_scaled = self.scaler_cov.fit_transform(self.covariates)

        self.train_series, self.test_series = self.series_scaled.split_after(train_ratio)
        self.train_cov, self.test_cov = self.covariates_scaled.split_after(train_ratio)

    def train_model(self):
        self.model = TransformerModel(
            input_chunk_length=48,
            output_chunk_length=12,
            d_model=64,
            nhead=4,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dropout=0.15,
            batch_size=32,
            n_epochs=50,
            nr_epochs_val_period=1,
            model_name="oil-temp-final",
            random_state=42
        )
        self.model.fit(series=self.train_series, past_covariates=self.train_cov, verbose=True)

    def predict_and_evaluate(self):
        pred = self.model.predict(n=len(self.test_series), series=self.train_series, past_covariates=self.covariates_scaled)
        self.pred_inv = self.scaler_series.inverse_transform(pred)
        self.test_inv = self.scaler_series.inverse_transform(self.test_series)

        print(f"✅ RMSE: {rmse(self.test_inv, self.pred_inv):.2f}")
        print(f"✅ MAE: {mae(self.test_inv, self.pred_inv):.2f}")

    def plot_one_day(self):
        df_test = pd.DataFrame({'oil temperature': self.test_inv.values().flatten()}, index=self.test_inv.time_index)
        df_pred = pd.DataFrame({'oil temperature': self.pred_inv.values().flatten()}, index=self.pred_inv.time_index)

        start_time = df_test.index[0].replace(hour=0, minute=0, second=0)
        end_time = start_time + pd.Timedelta(hours=23)

        df_test_day = df_test[(df_test.index >= start_time) & (df_test.index <= end_time)]
        df_pred_day = df_pred[(df_pred.index >= start_time) & (df_pred.index <= end_time)]

        plt.figure(figsize=(6, 3))
        plt.plot(df_test_day.index, df_test_day['oil temperature'], label='Actual', color='blue')
        plt.plot(df_pred_day.index, df_pred_day['oil temperature'], label='Predicted', color='orange')
        plt.legend()
        plt.title(f"Actual vs Predicted Oil Temperature\n{start_time.date()} 00:00 to 23:00")
        plt.xlabel("Time")
        plt.ylabel("Temperature")
        plt.grid(True)
        plt.tight_layout()
        plt.show()


c:\Users\pishva_r\Anaconda3\envs\test2\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
pipeline = OilTempForecastPipeline(file_path="main_trans.xlsx")
pipeline.load_and_preprocess()
pipeline.build_series()
pipeline.scale_and_split()
pipeline.train_model()
pipeline.predict_and_evaluate()
pipeline.plot_one_day()


FileNotFoundError: [Errno 2] No such file or directory: 'main_trans.xlsx'